In [1]:
import math
import soundfile as sf
import datetime
import numpy as np
from research.dataImporting import dataTools
import librosa
from librosa.feature import mfcc
import librosa
import pandas as pd
from librosa.effects import pitch_shift
import os
import pathlib
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from librosa.feature import mfcc,melspectrogram
import librosa.display

# This file is the backbone of all dataset generation for sUAS range estimation and detection. Creates datasets that are used for ML training and truth data for evaluation

# Code block creates MFCC dataset for ML training. Parses .wav files into MFCC frames and associates cellphone range (in meters) to sUAS for each frame

In [5]:
#Set Filepaths
parent_directory=os.path.abspath(os.path.join(os.getcwd(), ".."))

audio_file_path: str= "C:\\Users\\rclendening\\researchData\\EscapeCell_DataWav_V2\\"

redVox_file_path: str ="C:\\Users\\rclendening\\researchData\\Unused_Datasets\\EscapeCell_Data\\"

sUAS_GPS_file_path: str =parent_directory+"\\flight_directories\\UAS_log_files\\A1_A2_A3\\"

parent_directory=os.path.abspath(os.path.join(os.getcwd(), ".."))
passes_df=pd.read_csv(parent_directory+"\\flight_directories\\A1_A2_A3_flight_directory_w_Matrice_Phantom.csv")



In [7]:
def convertToGPS(path,name):
    window = dataTools.import_redVoxData(path)
    station = window.first_station()

    locationSensor = station.best_location_sensor()
    if locationSensor:
        lat=locationSensor._data.columns[3]
        lon=locationSensor._data.columns[4]
        alt=locationSensor._data.columns[5]
        phoneGPS.append([name,np.mean(lat),np.mean(lon),150.0]) #gps altitude seems a bit off
def searchDir(rootdir):
    it = 0
    for it in os.scandir(rootdir):
        if it.is_dir():
            # print(it.path)
            searchDir(it)
        else:
            it = os.path.dirname(it)
            break
    if isinstance(it, str):
        splitPath = it.split('\\')
        convertToGPS(it,splitPath[8])

def get_filepaths(data_path):
    data_files=[]
    for path, subdirs, files in os.walk(data_path):
        for name in files:
            data_files.append(os.path.join(path, name))
    return data_files



def create_dataset(train_files, totTime):
    '''
    Creates feature dataset and label dataset.
    @param train_files: EagerTensor of file paths.
    @return list of features (ds), list of labels corresponding to feature dataset:
    '''
    features = {}
    for x in train_files:
        #test_file = tf.io.read_file(x)
        #test_audio, sampleRate = tf.audio.decode_wav(contents=test_file)
        #test_audio, sampleRate = librosa.load(x, sr=8000)
        test_audio = sf.read(x)[0]
        x = str(x)
        if min(np.asarray(test_audio)) != 0 and len(test_audio)==totTime*8000 :
            #newData = test_audio[0: test_audio.shape[0] - test_audio.shape[0] % sampleRate]  # trim to nearest second
            #newFeats, newLabs = split_audio(test_audio, int(sampleRate))
            phone_features = MFCCCalc(test_audio.squeeze(),Fs=8000)
            features[x.split("\\")[7].split(".")[0]]=phone_features.transpose()
        # else:
        #     features.append([x.split("\\")[7].split(".")[0],0.00]) #just trying to fill space

    return features
def split_audio(audio_data,overlap,size,fs):
    frames=[]
    lead=size*fs
    follow=0
    hop=(overlap*size)*fs
    if overlap == 0:
        hop=size*fs
    while lead<=len(audio_data):
        frames.append(audio_data[int(follow):int(lead)])
        follow= follow+hop
        lead=lead+hop
    return frames
def create_spect_dataset(train_files, totTime):
    '''
    Creates feature dataset and label dataset.
    @param train_files: EagerTensor of file paths.
    @return list of features (ds), list of labels corresponding to feature dataset:
    '''
    features = {}
    raw_feats={}
    for x in train_files:
        phone_spects=[]
        phone_raw=[]
        #test_file = tf.io.read_file(x)
        #test_audio, sampleRate = tf.audio.decode_wav(contents=test_file)
        test_audio, sampleRate = librosa.load(x, sr=8000)
        x = str(x)
        if min(np.asarray(test_audio)) != 0 and len(test_audio)==totTime*8000 :
            #newData = test_audio[0: test_audio.shape[0] - test_audio.shape[0] % sampleRate]  # trim to nearest second
            #newFeats, newLabs = split_audio(test_audio, int(sampleRate))
            split=split_audio(test_audio,0.0,.256,8000)
            for frame in split:
                #spect = melspect(frame.squeeze(),Fs=8000)
                #framev2=frame
                #frame=np.int16((frame / np.max(np.abs(frame)) * 32767))
                # MFCC_data=MFCCCalc(frame,8000)
                # MFCC_data=MFCC_data.squeeze()/np.sum(MFCC_data.squeeze())
                phone_raw.append(frame)
                #spect=spect.flatten() #np.reshape(64,4) to unflatten
                #phone_spects.append(spect)
            #features[x.split("\\")[7].split(".")[0]]=np.array(phone_spects)
            raw_feats[x.split("\\")[7].split(".")[0]]=np.array(phone_raw)
        # else:
        #     features.append([x.split("\\")[7].split(".")[0],0.00]) #just trying to fill space

    return raw_feats
def split_audio(audio_data,overlap,size,fs):
    frames=[]
    lead=size*fs
    follow=0
    hop=(overlap*size)*fs
    if overlap == 0:
        hop=size*fs
    while lead<=len(audio_data):
        frames.append(audio_data[int(follow):int(lead)])
        follow= follow+hop
        lead=lead+hop
    return frames
def melspect(audioData, Fs):
    '''
    Converts decoded wav file to MFCC feature space
    @param audioData: Numpy array of decoded audio wav file
    @return MFCC coefficients
    '''
    data= audioData.astype(float)
    spect = melspectrogram(y=data,sr=Fs, hop_length=1024,n_mels=64, n_fft=2048)
    fig, ax = plt.subplots()
    S_dB = librosa.power_to_db(spect, ref=np.max)
    img = librosa.display.specshow(S_dB, x_axis='time',
                             y_axis='mel', sr=8000,
                             fmax=4000, ax=ax)
    fig.colorbar(img, ax=ax, format='%+2.0f dB')
    ax.set(title='Mel-frequency spectrogram')
    return spect

def MFCCCalc(audioData, Fs):
    '''
    Converts decoded wav file to MFCC feature space
    @param audioData: Numpy array of decoded audio wav file
    @return MFCC coefficients
    '''
    #audioData=audioData.numpy()
    data= audioData.astype(float)
    #coefs = mfcc(data, sr=sampleRate, hop_length=2048)
    coefs = mfcc(y=data, hop_length=2048,n_fft=2048,n_mfcc=40, sr=Fs)
    #coefs = mfcc(y=data, hop_length=2049,n_fft=2049,n_mfcc=40, sr=Fs)
    return coefs

to_meters=111139
range_df=pd.DataFrame(index=range(0),columns=(range(47)))
#range_df= pd.DataFrame(index=range(0),columns=(range(263)))
range_raw_df= pd.DataFrame(index=range(0),columns=(range(7)))
#range_df=pd.DataFrame()
#test123=readDirectoryCSV(r"C:\Users\rclendening\researchData\researchCSVs_Scripts_etc\A1_A2_flight_directory.csv")
def distanceCalc(phoneLat,phoneLon,phoneAlt, droneLat,droneLon,droneAlt):
    lat_delta= (phoneLat-droneLat)*to_meters
    lon_delta=(phoneLon-droneLon)*to_meters
    return np.sqrt((lat_delta**2+lon_delta**2+(phoneAlt-droneAlt)**2))


def maxValueRanks(maxValArray):
    maxValueRankArr=[]
    for x in maxValArray:
        seq = sorted(x)
        index = [seq.index(v) for v in x]
        maxValueRankArr.append(index)
    return maxValueRankArr

for x in passes_df.iterrows():
    indiv_df= pd.DataFrame(index=range(0),columns=(range(47)))
    phoneGPS=[]
    row=x[1]
    pass_num=row[0]
    scenario=row[1]
    run_num=row[2]
    start=row[3]
    stop=row[4]
    drone_gps_file=row[5]
    full_name=(str(scenario) + 'R' + str(run_num) + 'P' + str(pass_num)).strip()
    searchDir(redVox_file_path+scenario+"\\"+full_name)
    GPS_df=pd.DataFrame(phoneGPS, columns=['Name','Lat','Lon','Alt'])

    start_time=datetime.datetime(2021, 8, int(start[6:8]), int(start[9:11]), int(start[11:13]),int(start[13:15])).timestamp()
    stop_time=datetime.datetime(2021, 8, int(stop[6:8]), int(stop[9:11]), int(stop[11:13]),int(stop[13:15])).timestamp()
    if drone_gps_file[0]== 'F':
        FLY=True
    else:
        FLY=False
    GPS_flight_log=pd.read_csv((sUAS_GPS_file_path+drone_gps_file+".csv"))
    print(sUAS_GPS_file_path+drone_gps_file+".csv")
    if FLY:
        lon=GPS_flight_log.iloc[:,4]
        offsets=GPS_flight_log.iloc[:,1]
        lat=GPS_flight_log.iloc[:,5]
        time=GPS_flight_log.iloc[:,7]
        date=GPS_flight_log.iloc[:,8]
        height=GPS_flight_log.iloc[:,9]
    else:
        lon=GPS_flight_log.iloc[:,3]/10**7
        lat=GPS_flight_log.iloc[:,2]/10**7
        time_UTC=GPS_flight_log.iloc[:,1]/10**6
        height=GPS_flight_log.iloc[:,4]/10**3
    usable_phones=[]
    tot_time= stop_time-start_time

    filepaths= get_filepaths((audio_file_path+scenario+"\\"+full_name))
    phone_features= create_dataset(filepaths,tot_time)
    #phone_features= create_spect_dataset(filepaths,tot_time)
    first =True
    for ph in phoneGPS:
        if 43.00 < ph[1] < 44.00 and -75.00 > ph[2] > -76.00 and ph[0] in phone_features:
            usable_phones.append(ph)
    tot_segs = len(phone_features[list(phone_features.keys())[0]])
    t=0
    done=False
    MFCC_index=0
    while not done:
        if FLY:
            if np.isnan(time[t]):
                cur_time=0
            else:
                if first:
                    offset=offsets[t]
                    first=False
                    offset_time=datetime.datetime(2021, 8, int(start[6:8]), int(str(time[t])[0:2])-4,int(str(time[t])[2:4]),int(str(time[t])[4:6])).timestamp()
                cur_time=offset_time+abs(offsets[t]-offset)
        else:
            cur_time=time_UTC[t]

        time_delta=cur_time-start_time
        prev_index=MFCC_index
        MFCC_index = math.floor(tot_segs*time_delta/tot_time)
        if  start_time <= cur_time< stop_time and MFCC_index!= prev_index:
            #MFCC_df=pd.DataFrame()
            for phone in usable_phones:
                val = distanceCalc(phone[1],phone[2],phone[3], lat[t],lon[t],height[t])
                # range_df.loc[len(range_df.index)]=[str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val),pd.Series(phone_features[phone[0]][MFCC_index])]

                #data=[full_name,str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val),start_time,cur_time]

                data=[full_name,str(MFCC_index),str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val)]
                data.extend(phone_features[phone[0]][MFCC_index].tolist()) #MFCCs
                # data_raw=[full_name,str(MFCC_index),str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val)]
                # data_raw.extend(phone_features[phone[0]][MFCC_index].tolist()) #MFCCs
                indiv_df.loc[len(indiv_df.index)]=data
                #indiv_raw_df.loc[len(indiv_raw_df.index)]=data_raw
        t+=4
        if cur_time>stop_time:
            done=True

    range_df= pd.concat([range_df, indiv_df])



def rename_bulk_columns(DF,nameList):
    for x in range(len(nameList)):
        DF.columns.values[x]=nameList[x]

column_names=["Pass","Segment","Name","Lat","Lon","Alt","Range","MFCC_vals(0-40)"]
range_df=rename_bulk_columns(range_df,column_names)

range_df.to_csv('LT80m_detection_training_w_matrice_phantom_unplayable.csv')


KeyboardInterrupt



# Raw Audio Dataset Creator

In [7]:
# Set raw dataset parameters
frame_length=0.5
overlap=0
Fs=8000

In [13]:
def create_raw_dataset(train_files, totTime,frame_length,overlap):
    '''
    Creates feature dataset and label dataset.
    @param train_files: EagerTensor of file paths.
    @return list of features (ds), list of labels corresponding to feature dataset:
    '''
    raw_feats={}
    for x in train_files:
        phone_raw=[]
        test_audio, sampleRate = librosa.load(x, sr=Fs)
        x = str(x)
        if min(np.asarray(test_audio)) != 0 and len(test_audio)==totTime*Fs :
            split=split_audio(test_audio,overlap,frame_length,Fs)
            for frame in split:
                phone_raw.append(frame)
            raw_feats[x.split("\\")[7].split(".")[0]]=np.array(phone_raw)
    return raw_feats

frame_length=0.5
dictionary_list=[]
to_meters=111139
range_df=pd.DataFrame(index=range(0),columns=(range(int((frame_length*Fs)+7))))

range_raw_df=pd.DataFrame(index=range(0),columns=(range(int((frame_length*Fs)+7))))

for x in passes_df.iterrows():
    #indiv_df=pd.DataFrame(columns=["Pass","Segment","Name","Lat","Lon","Alt","Range",'MFCC_0'])
    indiv_df= pd.DataFrame(index=range(0),columns=(range(int((frame_length*Fs)+7))))
    #indiv_raw_df= pd.DataFrame(index=range(0),columns=(range(2055)))
    phoneGPS=[]
    row=x[1]
    pass_num=row[0]
    scenario=row[1]
    run_num=row[2]
    start=row[3]
    stop=row[4]
    drone_gps_file=row[5]
    full_name=(str(scenario) + 'R' + str(run_num) + 'P' + str(pass_num)).strip()
    searchDir(redVox_file_path+scenario+"\\"+full_name)
    GPS_df=pd.DataFrame(phoneGPS, columns=['Name','Lat','Lon','Alt'])

    start_time=datetime.datetime(2021, 8, int(start[6:8]), int(start[9:11]), int(start[11:13]),int(start[13:15])).timestamp()
    stop_time=datetime.datetime(2021, 8, int(stop[6:8]), int(stop[9:11]), int(stop[11:13]),int(stop[13:15])).timestamp()
    if drone_gps_file[0]== 'F':
        FLY=True
    else:
        FLY=False
    GPS_flight_log=pd.read_csv((sUAS_GPS_file_path+drone_gps_file+".csv"))
    print(sUAS_GPS_file_path+drone_gps_file+".csv")
    if FLY:
        lon=GPS_flight_log.iloc[:,4]
        offsets=GPS_flight_log.iloc[:,1]
        lat=GPS_flight_log.iloc[:,5]
        time=GPS_flight_log.iloc[:,7]
        date=GPS_flight_log.iloc[:,8]
        height=GPS_flight_log.iloc[:,9]
    else:
        lon=GPS_flight_log.iloc[:,3]/10**7
        lat=GPS_flight_log.iloc[:,2]/10**7
        time_UTC=GPS_flight_log.iloc[:,1]/10**6
        height=GPS_flight_log.iloc[:,4]/10**3

    usable_phones=[]
    tot_time= stop_time-start_time
    filepaths= get_filepaths((audio_file_path+scenario+"\\"+full_name))
    phone_features= create_raw_dataset(filepaths, tot_time,frame_length,overlap)
    first =True
    for ph in phoneGPS:
        if 43.00 < ph[1] < 44.00 and -75.00 > ph[2] > -76.00 and ph[0] in phone_features:
            usable_phones.append(ph)
    tot_segs = len(phone_features[list(phone_features.keys())[0]])
    t=0
    done=False
    MFCC_index=0
    while not done:
        if FLY:
            if np.isnan(time[t]):
                cur_time=0
            else:
                if first:
                    offset=offsets[t]
                    first=False
                    offset_time=datetime.datetime(2021, 8, int(start[6:8]), int(str(time[t])[0:2])-4,int(str(time[t])[2:4]),int(str(time[t])[4:6])).timestamp()
                cur_time=offset_time+abs(offsets[t]-offset)
        else:
            cur_time=time_UTC[t]

        time_delta=cur_time-start_time
        prev_index=MFCC_index
        MFCC_index = math.floor(tot_segs*time_delta/tot_time)
        if  start_time <= cur_time< stop_time and MFCC_index!= prev_index:
            #MFCC_df=pd.DataFrame()
            for phone in usable_phones:
                val = distanceCalc(phone[1],phone[2],phone[3], lat[t],lon[t],height[t])
                # range_df.loc[len(range_df.index)]=[str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val),pd.Series(phone_features[phone[0]][MFCC_index])]

                #data=[full_name,str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val),start_time,cur_time]

                data=[full_name,str(MFCC_index),str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val)]
                data.extend(phone_features[phone[0]][MFCC_index].tolist()) #MFCCs
                # data_raw=[full_name,str(MFCC_index),str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val)]
                # data_raw.extend(phone_features[phone[0]][MFCC_index].tolist()) #MFCCs
                indiv_df.loc[len(indiv_df.index)]=data
                #indiv_raw_df.loc[len(indiv_raw_df.index)]=data_raw
        t+=4
        if cur_time>stop_time:
            done=True

    range_df= pd.concat([range_df, indiv_df])
column_names=["Pass","Segment","Name","Lat","Lon","Alt","Range","raw_audio"]
range_df=rename_bulk_columns(range_df,column_names)

range_df.to_csv('0.5_raw_audio_ranges.csv')

KeyboardInterrupt: 

Data augmentation is a failed effort... does not improve overfitting issues what so ever. In EVP, the ML model that uses data augmentation achieves 0.74 F1 score for EVP and 0.55 Phones.

Currently edited to include peak normalization. On each training sample

In [12]:
def rename_bulk_columns(DF,nameList):
    new_df=DF
    for x in range(len(nameList)):
        new_df=new_df.rename(columns={DF.columns[x]:nameList[x]})
    return new_df
column_names=["Pass","Segment","Name","Lat","Lon","Alt","Range","MFCC_vals(0-40)"]

range_df_1=rename_bulk_columns(range_df,column_names)

range_df_1.to_csv('0.5_raw_audio_ranges.csv')

KeyboardInterrupt: 

# Frequency Shift Dataset Augmentation (Up-samp Down-samp) / Changed to noise mixed

In [28]:
def convertToGPS(path,name):
    window = dataTools.import_redVoxData(path)
    station = window.first_station()

    locationSensor = station.best_location_sensor()
    if locationSensor:
        lat=locationSensor._data.columns[3]
        lon=locationSensor._data.columns[4]
        alt=locationSensor._data.columns[5]
        phoneGPS.append([name,np.mean(lat),np.mean(lon),150.0]) #gps altitude seems a bit off

noise_path=r"C:\Users\rclendening\researchData\YT_vids\Noise"
def searchDir(rootdir):
    it = 0
    for it in os.scandir(rootdir):
        if it.is_dir():
            # print(it.path)
            searchDir(it)
        else:
            it = os.path.dirname(it)
            break
    if isinstance(it, str):
        splitPath = it.split('\\')
        convertToGPS(it,splitPath[8])

def get_filepaths(data_path):
    data_files=[]
    for path, subdirs, files in os.walk(data_path):
        for name in files:
            data_files.append(os.path.join(path, name))
    return data_files

# def split_audio(waveData, sampleFreq):
#     '''
#     Frames audio data and converts to feature space (MFCC)
#     :param waveData: waveData array of time-domain audio
#     :param sampleFreq: Sample Frequency (8Khz)
#     @return list of features (ds), list of labels corresponding to feature dataset:
#     '''
#     # middle third of data
#     duration = waveData.shape[0]
#     startTime = np.round(duration / 3)
#     endTime = np.round(duration * 2 / 3)
#     waveDataSplit= waveData[int(startTime):int(endTime)]
#     features=MFCCCalc(waveDataSplit.squeeze(), sampleFreq)
#     #label= [droneDict[labelName]] * features.shape[1]
#     return features

def create_dataset(train_files, totTime):
    '''
    Creates feature dataset and label dataset.
    @param train_files: EagerTensor of file paths.
    @return list of features (ds), list of labels corresponding to feature dataset:
    '''
    features = {}
    for x in train_files:
        #test_file = tf.io.read_file(x)
        #test_audio, sampleRate = tf.audio.decode_wav(contents=test_file)
        test_audio, sampleRate = librosa.load(x, sr=8000)
        x = str(x)
        if min(np.asarray(test_audio)) != 0 and len(test_audio)==totTime*8000 :
            #newData = test_audio[0: test_audio.shape[0] - test_audio.shape[0] % sampleRate]  # trim to nearest second
            #newFeats, newLabs = split_audio(test_audio, int(sampleRate))
            phone_features = MFCCCalc(test_audio.squeeze(),Fs=8000)
            features[x.split("\\")[7].split(".")[0]]=phone_features.transpose()
        # else:
        #     features.append([x.split("\\")[7].split(".")[0],0.00]) #just trying to fill space

    return features

def create_aug_dataset(train_files, totTime, fs, noise_files):
    '''
    Creates feature dataset and label dataset.
    @param train_files: EagerTensor of file paths.
    @return list of features (ds), list of labels corresponding to feature dataset:
    '''
    features = {}
    noise_audio_file=noise_files[np.random.randint(0,2)]
    noise_audio, sampleRate = librosa.load(noise_audio_file, sr=fs)
    for x in train_files:
        #test_file = tf.io.read_file(x)
        #test_audio, sampleRate = tf.audio.decode_wav(contents=test_file)
        test_audio, sampleRate = librosa.load(x, sr=fs)
        x = str(x)
        if min(np.asarray(test_audio)) != 0 and len(test_audio)==totTime*8000 :
            noise_audio=np.resize(noise_audio,len(test_audio))
            test_audio=test_audio * np.random.uniform(0.8, 1.2) + noise_audio * np.random.uniform(0, 0.1)
            #newData = test_audio[0: test_audio.shape[0] - test_audio.shape[0] % sampleRate]  # trim to nearest second
            #newFeats, newLabs = split_audio(test_audio, int(sampleRate))
            phone_features = MFCCCalc(test_audio.squeeze(),Fs=8000)
            features[x.split("\\")[7].split(".")[0]]=phone_features.transpose()
        # else:
        #     features.append([x.split("\\")[7].split(".")[0],0.00]) #just trying to fill space

    return features
def MFCCCalc(audioData, Fs):
    '''
    Converts decoded wav file to MFCC feature space
    @param audioData: Numpy array of decoded audio wav file
    @return MFCC coefficients
    '''
    #audioData=audioData.numpy()
    data= audioData.astype(float)
    #coefs = mfcc(data, sr=sampleRate, hop_length=2048)
    coefs = mfcc(y=data, hop_length=2048,n_fft=2048,n_mfcc=40, sr=Fs)
    return coefs
aug_vals=[7000,7200,7400,7600,7800,8000,8200,8400,8600,8800,9000]
dictionary_list=[]
passes_df=pd.read_csv(r"C:\Users\rclendening\researchData\researchCSVs_Scripts_etc\A1_A2_A3_flight_directory_w_Matrice_Phantom.csv")
print()
to_meters=111139
range_df=pd.DataFrame(columns=["Pass","Segment","Name","Lat","Lon","Alt","Range"])
#range_df= pd.DataFrame(index=range(0),columns=(range(263)))
range_raw_df= pd.DataFrame(index=range(0),columns=(range(7)))
#range_df=pd.DataFrame()
#test123=readDirectoryCSV(r"C:\Users\rclendening\researchData\researchCSVs_Scripts_etc\A1_A2_flight_directory.csv")
def distanceCalc(phoneLat,phoneLon,phoneAlt, droneLat,droneLon,droneAlt):
    lat_delta= (phoneLat-droneLat)*to_meters
    lon_delta=(phoneLon-droneLon)*to_meters
    return np.sqrt((lat_delta**2+lon_delta**2+(phoneAlt-droneAlt)**2))


def maxValueRanks(maxValArray):
    maxValueRankArr=[]
    for x in maxValArray:
        seq = sorted(x)
        index = [seq.index(v) for v in x]
        maxValueRankArr.append(index)
    return maxValueRankArr

for x in passes_df.iterrows():
    #indiv_df=pd.DataFrame(columns=["Pass","Segment","Name","Lat","Lon","Alt","Range",'MFCC_0'])
    #indiv_df= pd.DataFrame(index=range(0),columns=(range(263)))
    indiv_raw_df= pd.DataFrame(index=range(0),columns=(range(47)))
    phoneGPS=[]
    row=x[1]
    pass_num=row[0]
    scenario=row[1]
    run_num=row[2]
    start=row[3]
    stop=row[4]
    drone_gps_file=row[5]
    full_name=(str(scenario) + 'R' + str(run_num) + 'P' + str(pass_num)).strip()
    if scenario =="A1" or scenario=="A2":
        indiv_raw_df= pd.DataFrame(index=range(0),columns=(range(47)))
        searchDir(("C:\\Users\\rclendening\\researchData\\Unused_Datasets\\EscapeCell_Data\\"+scenario+"\\"+full_name))
        GPS_df=pd.DataFrame(phoneGPS, columns=['Name','Lat','Lon','Alt'])
        start_time=datetime.datetime(2021, 8, int(start[6:8]), int(start[9:11]), int(start[11:13]),int(start[13:15])).timestamp()
        stop_time=datetime.datetime(2021, 8, int(stop[6:8]), int(stop[9:11]), int(stop[11:13]),int(stop[13:15])).timestamp()
        if drone_gps_file[0]== 'F':
            FLY=True
        else:
            FLY=False
        GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))
        print("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv")
        if FLY:
            lon=GPS_flight_log.iloc[:,4]
            offsets=GPS_flight_log.iloc[:,1]
            lat=GPS_flight_log.iloc[:,5]
            time=GPS_flight_log.iloc[:,7]
            date=GPS_flight_log.iloc[:,8]
            height=GPS_flight_log.iloc[:,9]
        else:
            lon=GPS_flight_log.iloc[:,3]/10**7
            lat=GPS_flight_log.iloc[:,2]/10**7
            time_UTC=GPS_flight_log.iloc[:,1]/10**6
            #height=GPS_flight_log.iloc[:,5]/10**3
            height=GPS_flight_log.iloc[:,4]/10**3
        #for phones in GPS_df: #FLY files Long=col(5) lat=col(6) date=col(7) time=col(8) height (MSL)=col(10)
                            #Non-FLY files lat=col(3) lon=col(4) timestamp(UTC)=col(1) height (MSL)= col(5) in millimeters
        usable_phones=[]
        tot_time= stop_time-start_time
        noise_filepaths= get_filepaths(noise_path)
        filepaths= get_filepaths("C:\\Users\\rclendening\\researchData\\EscapeCell_DataWav_V2\\"+scenario+"\\"+full_name)
        phone_features = create_aug_dataset(filepaths,tot_time,8000,noise_filepaths)
        #phone_features, phone_raw= create_dataset(filepaths,tot_time)
        first =True
        for ph in phoneGPS:
            if 43.00 < ph[1] < 44.00 and -75.00 > ph[2] > -76.00 and ph[0] in phone_features:
                usable_phones.append(ph)
        tot_segs = len(phone_features[list(phone_features.keys())[0]])
        #tot_segs=tot_time/.128
        # for t in range(0,len(GPS_flight_log),10):
        t=0
        done=False
        MFCC_index=0
        while not done:
            if FLY:
                if np.isnan(time[t]):
                    cur_time=0
                else:
                    if first:
                        offset=offsets[t]
                        first=False
                        offset_time=datetime.datetime(2021, 8, int(start[6:8]), int(str(time[t])[0:2])-4,int(str(time[t])[2:4]),int(str(time[t])[4:6])).timestamp()
                    cur_time=offset_time+abs(offsets[t]-offset)
            else:
                cur_time=time_UTC[t]

            time_delta=cur_time-start_time
            prev_index=MFCC_index
            MFCC_index = math.floor(tot_segs*time_delta/tot_time)
            if  start_time <= cur_time< stop_time and MFCC_index!= prev_index:
                #MFCC_df=pd.DataFrame()
                for phone in usable_phones:
                    val = distanceCalc(phone[1],phone[2],phone[3], lat[t],lon[t],height[t])
                    # range_df.loc[len(range_df.index)]=[str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val),pd.Series(phone_features[phone[0]][MFCC_index])]

                    #data=[full_name,str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val),start_time,cur_time]

                    #data=[full_name,str(MFCC_index),str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val)]
                    #data.extend(phone_features[phone[0]][MFCC_index].tolist()) #MFCCs
                    #data_raw=[full_name+"_"+str(x).strip(),str(MFCC_index),str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val)]
                    data_raw=[full_name,str(MFCC_index),str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val)]
                    data_raw.extend(phone_features[phone[0]][MFCC_index].tolist()) #MFCCs
                    #indiv_df.loc[len(indiv_df.index)]=data
                    indiv_raw_df.loc[len(indiv_raw_df.index)]=data_raw
            t+=4
            if cur_time>stop_time:
                done=True

        #range_df= pd.concat([range_df, indiv_df])
        range_raw_df=pd.concat([range_raw_df,indiv_raw_df])
    else:
        searchDir(("C:\\Users\\rclendening\\researchData\\Unused_Datasets\\EscapeCell_Data\\"+scenario+"\\"+full_name))
        GPS_df=pd.DataFrame(phoneGPS, columns=['Name','Lat','Lon','Alt'])
        start_time=datetime.datetime(2021, 8, int(start[6:8]), int(start[9:11]), int(start[11:13]),int(start[13:15])).timestamp()
        stop_time=datetime.datetime(2021, 8, int(stop[6:8]), int(stop[9:11]), int(stop[11:13]),int(stop[13:15])).timestamp()
        if drone_gps_file[0]== 'F':
            FLY=True
        else:
            FLY=False
        GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))
        print("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv")
        if FLY:
            lon=GPS_flight_log.iloc[:,4]
            offsets=GPS_flight_log.iloc[:,1]
            lat=GPS_flight_log.iloc[:,5]
            time=GPS_flight_log.iloc[:,7]
            date=GPS_flight_log.iloc[:,8]
            height=GPS_flight_log.iloc[:,9]
        else:
            lon=GPS_flight_log.iloc[:,3]/10**7
            lat=GPS_flight_log.iloc[:,2]/10**7
            time_UTC=GPS_flight_log.iloc[:,1]/10**6
            #height=GPS_flight_log.iloc[:,5]/10**3
            height=GPS_flight_log.iloc[:,4]/10**3
        #for phones in GPS_df: #FLY files Long=col(5) lat=col(6) date=col(7) time=col(8) height (MSL)=col(10)
                            #Non-FLY files lat=col(3) lon=col(4) timestamp(UTC)=col(1) height (MSL)= col(5) in millimeters
        usable_phones=[]
        tot_time= stop_time-start_time

        filepaths= get_filepaths("C:\\Users\\rclendening\\researchData\\EscapeCell_DataWav_V2\\"+scenario+"\\"+full_name)
        phone_features= create_dataset(filepaths,tot_time)
        #phone_features, phone_raw= create_dataset(filepaths,tot_time)
        first =True
        for ph in phoneGPS:
            if 43.00 < ph[1] < 44.00 and -75.00 > ph[2] > -76.00 and ph[0] in phone_features:
                usable_phones.append(ph)
        tot_segs = len(phone_features[list(phone_features.keys())[0]])
        #tot_segs=tot_time/.128
        # for t in range(0,len(GPS_flight_log),10):
        t=0
        done=False
        MFCC_index=0
        while not done:
            if FLY:
                if np.isnan(time[t]):
                    cur_time=0
                else:
                    if first:
                        offset=offsets[t]
                        first=False
                        offset_time=datetime.datetime(2021, 8, int(start[6:8]), int(str(time[t])[0:2])-4,int(str(time[t])[2:4]),int(str(time[t])[4:6])).timestamp()
                    cur_time=offset_time+abs(offsets[t]-offset)
            else:
                cur_time=time_UTC[t]

            time_delta=cur_time-start_time
            prev_index=MFCC_index
            MFCC_index = math.floor(tot_segs*time_delta/tot_time)
            if  start_time <= cur_time< stop_time and MFCC_index!= prev_index:
                #MFCC_df=pd.DataFrame()
                for phone in usable_phones:
                    val = distanceCalc(phone[1],phone[2],phone[3], lat[t],lon[t],height[t])
                    # range_df.loc[len(range_df.index)]=[str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val),pd.Series(phone_features[phone[0]][MFCC_index])]

                    #data=[full_name,str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val),start_time,cur_time]

                    #data=[full_name,str(MFCC_index),str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val)]
                    #data.extend(phone_features[phone[0]][MFCC_index].tolist()) #MFCCs
                    data_raw=[full_name,str(MFCC_index),str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val)]
                    data_raw.extend(phone_features[phone[0]][MFCC_index].tolist()) #MFCCs
                    #indiv_df.loc[len(indiv_df.index)]=data
                    indiv_raw_df.loc[len(indiv_raw_df.index)]=data_raw
            t+=4
            if cur_time>stop_time:
                done=True
        #range_df= pd.concat([range_df, indiv_df])
        range_raw_df=pd.concat([range_raw_df,indiv_raw_df])
#range_df.to_csv('0.5s_spec_range_A1_A2_A3_no_overlap.csv')
range_raw_df.to_csv('LT80m_detection_training_w_matrice_phantom_augment_A1_A2_A3.csv')

C:\Users\rclendening\AppData\Local\Temp\1\ipykernel_17624\2243195096.py:154: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))


C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY101(1123-1155).csv


C:\Users\rclendening\AppData\Local\Temp\1\ipykernel_17624\2243195096.py:154: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))


C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY101(1123-1155).csv


C:\Users\rclendening\AppData\Local\Temp\1\ipykernel_17624\2243195096.py:154: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))


...

C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY135.csv


C:\Users\rclendening\AppData\Local\Temp\1\ipykernel_17624\2243195096.py:233: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))


C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY135.csv


C:\Users\rclendening\AppData\Local\Temp\1\ipykernel_17624\2243195096.py:233: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))


C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY135.csv
C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY036.csv


In [36]:
column_names=["Pass","Segment","Name","Lat","Lon","Alt","Range","MFCC_vals(0-40)"]
def rename_bulk_columns(DF,nameList):
    for x in range(len(nameList)):
        DF.columns.values[x]=nameList[x]

range_raw_df.to_csv('LT80m_detection_training_w_matrice_phantom_augment_A1_A2_A3.csv')

# Just ranges

In [11]:
for x in passes_df.iterrows():
    #indiv_df=pd.DataFrame(columns=["Pass","Segment","Name","Lat","Lon","Alt","Range",'MFCC_0'])
    #indiv_df= pd.DataFrame(index=range(0),columns=(range(263)))
    indiv_raw_df= pd.DataFrame(index=range(0),columns=(range(7)))
    phoneGPS=[]
    row=x[1]
    pass_num=row[0]
    scenario=row[1]
    run_num=row[2]
    start=row[3]
    stop=row[4]
    drone_gps_file=row[5]
    full_name=(str(scenario) + 'R' + str(run_num) + 'P' + str(pass_num)).strip()

    searchDir(("C:\\Users\\rclendening\\researchData\\Unused_Datasets\\EscapeCell_Data\\"+scenario+"\\"+full_name))
    GPS_df=pd.DataFrame(phoneGPS, columns=['Name','Lat','Lon','Alt'])

    start_time=datetime.datetime(2021, 8, int(start[6:8]), int(start[9:11]), int(start[11:13]),int(start[13:15])).timestamp()
    stop_time=datetime.datetime(2021, 8, int(stop[6:8]), int(stop[9:11]), int(stop[11:13]),int(stop[13:15])).timestamp()
    if drone_gps_file[0]== 'F':
        FLY=True
    else:
        FLY=False
    GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))
    print("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv")
    if FLY:
        lon=GPS_flight_log.iloc[:,4]
        offsets=GPS_flight_log.iloc[:,1]
        lat=GPS_flight_log.iloc[:,5]
        time=GPS_flight_log.iloc[:,7]
        date=GPS_flight_log.iloc[:,8]
        height=GPS_flight_log.iloc[:,9]
    else:
        lon=GPS_flight_log.iloc[:,3]/10**7
        lat=GPS_flight_log.iloc[:,2]/10**7
        time_UTC=GPS_flight_log.iloc[:,1]/10**6
        #height=GPS_flight_log.iloc[:,5]/10**3
        height=GPS_flight_log.iloc[:,4]/10**3
    #for phones in GPS_df: #FLY files Long=col(5) lat=col(6) date=col(7) time=col(8) height (MSL)=col(10)
                        #Non-FLY files lat=col(3) lon=col(4) timestamp(UTC)=col(1) height (MSL)= col(5) in millimeters
    usable_phones=[]
    tot_time= stop_time-start_time

    filepaths= get_filepaths("C:\\Users\\rclendening\\researchData\\EscapeCell_DataWav_V2\\"+scenario+"\\"+full_name)
    phone_features= create_dataset(filepaths,tot_time)
    #phone_features, phone_raw= create_dataset(filepaths,tot_time)
    first =True
    for ph in phoneGPS:
        if 43.00 < ph[1] < 44.00 and -75.00 > ph[2] > -76.00 and ph[0] in phone_features:
            usable_phones.append(ph)
    tot_segs = len(phone_features[list(phone_features.keys())[0]])
    # for t in range(0,len(GPS_flight_log),10):
    t=0
    done=False
    MFCC_index=0
    while not done:
        if FLY:
            if np.isnan(time[t]):
                cur_time=0
            else:
                if first:
                    offset=offsets[t]
                    first=False
                    offset_time=datetime.datetime(2021, 8, int(start[6:8]), int(str(time[t])[0:2])-4,int(str(time[t])[2:4]),int(str(time[t])[4:6])).timestamp()
                cur_time=offset_time+abs(offsets[t]-offset)
        else:
            cur_time=time_UTC[t]

        time_delta=cur_time-start_time
        prev_index=MFCC_index
        MFCC_index = math.floor(tot_segs*time_delta/tot_time)
        if  start_time <= cur_time< stop_time and MFCC_index!= prev_index:
            for phone in usable_phones:
                val = distanceCalc(phone[1],phone[2],phone[3], lat[t],lon[t],height[t])
                data_raw=[full_name,str(MFCC_index),str(phone[0]),np.float64(phone[1]),np.float64(phone[2]),np.float64(phone[3]),np.float64(val)]
                indiv_raw_df.loc[len(indiv_raw_df.index)]=data_raw
        t+=4
        if cur_time>stop_time:
            done=True
    range_raw_df=pd.concat([range_raw_df,indiv_raw_df])
range_raw_df.to_csv('A3_Range_truth_dot256ms.csv')

C:\Users\rclendening\AppData\Local\Temp\1\ipykernel_12432\473260165.py:189: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))


C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY131.csv


C:\Users\rclendening\AppData\Local\Temp\1\ipykernel_12432\473260165.py:189: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))


C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY131.csv


C:\Users\rclendening\AppData\Local\Temp\1\ipykernel_12432\473260165.py:189: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))


...

C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY037.csv
C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY036.csv
C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY036.csv


C:\Users\rclendening\AppData\Local\Temp\1\ipykernel_12432\473260165.py:189: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))


C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY037.csv


C:\Users\rclendening\AppData\Local\Temp\1\ipykernel_12432\473260165.py:189: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  GPS_flight_log=pd.read_csv(("C:\\Users\\rclendening\\researchData\\ESCAPE II_AFRL_SSD\\UAS Campaign\\UAS_log_files\\A1_A2_A3"+"\\"+drone_gps_file+".csv"))


C:\Users\rclendening\researchData\ESCAPE II_AFRL_SSD\UAS Campaign\UAS_log_files\A1_A2_A3\FLY037.csv
